In [1]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes
import nuscenes.lidarseg
from nuscenes.utils.data_classes import LidarPointCloud, RadarPointCloud, Box
import utils_data_viewer
from utils_data_viewer import view_scene_map, custom_draw_geometry, cam_vis, top_down, interactive_vis, visualize_lidar_in_img
from utils_data_viewer import project_points_in_img, colorise_pcd
#Radar
from utils_data_viewer import top_down_radar, interactive_vis_radar, radar_from_file, fuse_radars_in_ego, visualize_radar_in_img
import utils_obj_det
from utils_obj_det import get_radar_in_img, vis_obj_det_speed

import numpy as np
import os
import time
from scipy.spatial.transform import Rotation
from pyquaternion import Quaternion
import struct
import matplotlib as plt

import cv2
import open3d as o3d


nusc = NuScenes(version='v1.0-mini', dataroot='/data/sets/nuscenes', verbose=True)
utils_data_viewer.nusc = nusc
utils_obj_det.nusc = nusc

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.443 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [2]:
#nusc.list_scenes()
sc_id = 9

scene = nusc.scene[sc_id]
log = nusc.get('log', scene['log_token'])
map = nusc.get('map', log['map_token'])
map_file = os.path.join(nusc.dataroot, map['filename'])

first_sample = nusc.get('sample', scene['first_sample_token'])
last_sample = nusc.get('sample', scene['last_sample_token'])
sample = first_sample
cam_vis(scene, "CAM_FRONT", record=True)

### Record

In [ ]:
sc_ids = [0, 2, 3, 4, 6, 7, 8]
channels = ['CAM_FRONT', 'CAM_FRONT_LEFT', 'CAM_FRONT_RIGHT', 'CAM_BACK', 'CAM_BACK_LEFT', 'CAM_BACK_RIGHT']
for sc_i in sc_ids:
    scene = nusc.scene[sc_i]
    # for c in channels:
    #     cam_vis(scene, c, record=True)
    #     visualize_lidar_in_img(scene, c, record=True)
    #     visualize_radar_in_img(scene, c, record=True)
    #     vis_obj_det_speed(scene, cam_channel=c, record=True)
    # top_down(scene, record=True)
    #top_down_radar(scene, record=True)
    interactive_vis(scene, record=True)
    colorise_pcd(scene, record=True)
    interactive_vis_radar(scene, record=True)
    

# RADAR

In [ ]:
#nusc.render_pointcloud_in_image(sample['token'], pointsensor_channel='LIDAR_TOP')

In [ ]:
#Lidar
#view_scene_map(scene)
#visualize_lidar_in_img(scene, cam_channel='CAM_FRONT', record=True)
#colorise_pcd(scene, record=True)
cam_vis(scene, "CAM_FRONT")
#interactive_vis(scene, record=True)
#top_down(scene, record=True)

#Radar
#interactive_vis_radar(scene, record=True)
#top_down_radar(scene, record=True)
#visualize_radar_in_img(scene, cam_channel='CAM_BACK', record=True)

### BACKUP

In [ ]:
radar_channel = 'RADAR_FRONT'
radar = nusc.get('sample_data', sample['data'][radar_channel])
radar_file = os.path.join(nusc.dataroot, radar['filename'])
pts, fields = radar_from_file(radar_file)
radar_sensor = nusc.get('calibrated_sensor', radar['calibrated_sensor_token'])
velocities = pts[8:10, :]  # Compensated velocity

s, f = fuse_radars_in_ego(sample)
s.shape

velocities = np.vstack((velocities, np.zeros(pts.shape[1])))
pts[:3, 0]

In [ ]:
#This is basically just a copy from the radar pointcloud code from nuscenes devkit
def radar_from_file(radar_file):

    """
    Loads RADAR data from a Point Cloud Data file. See details below.
    :param file_name: The path of the pointcloud file.
    :param invalid_states: Radar states to be kept. See details below.
    :param dynprop_states: Radar states to be kept. Use [0, 2, 6] for moving objects only. See details below.
    :param ambig_states: Radar states to be kept. See details below.
    To keep all radar returns, set each state filter to range(18).
    :return: <np.float: d, n>. Point cloud matrix with d dimensions and n points.

    Example of the header fields:
    # .PCD v0.7 - Point Cloud Data file format
    VERSION 0.7
    FIELDS x y z dyn_prop id rcs vx vy vx_comp vy_comp is_quality_valid ambig_state x_rms y_rms invalid_state pdh0 vx_rms vy_rms
    SIZE 4 4 4 1 2 4 4 4 4 4 1 1 1 1 1 1 1 1
    TYPE F F F I I F F F F F I I I I I I I I
    COUNT 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
    WIDTH 125
    HEIGHT 1
    VIEWPOINT 0 0 0 1 0 0 0
    POINTS 125
    DATA binary

    Below some of the fields are explained in more detail:

    x is front, y is left

    vx, vy are the velocities in m/s.
    vx_comp, vy_comp are the velocities in m/s compensated by the ego motion.
    We recommend using the compensated velocities.

    invalid_state: state of Cluster validity state.
    (Invalid states)
    0x01	invalid due to low RCS
    0x02	invalid due to near-field artefact
    0x03	invalid far range cluster because not confirmed in near range
    0x05	reserved
    0x06	invalid cluster due to high mirror probability
    0x07	Invalid cluster because outside sensor field of view
    0x0d	reserved
    0x0e	invalid cluster because it is a harmonics
    (Valid states)
    0x00	valid
    0x04	valid cluster with low RCS
    0x08	valid cluster with azimuth correction due to elevation
    0x09	valid cluster with high child probability
    0x0a	valid cluster with high probability of being a 50 deg artefact
    0x0b	valid cluster but no local maximum
    0x0c	valid cluster with high artefact probability
    0x0f	valid cluster with above 95m in near range
    0x10	valid cluster with high multi-target probability
    0x11	valid cluster with suspicious angle

    dynProp: Dynamic property of cluster to indicate if is moving or not.
    0: moving
    1: stationary
    2: oncoming
    3: stationary candidate
    4: unknown
    5: crossing stationary
    6: crossing moving
    7: stopped

    ambig_state: State of Doppler (radial velocity) ambiguity solution.
    0: invalid
    1: ambiguous
    2: staggered ramp
    3: unambiguous
    4: stationary candidates

    pdh0: False alarm probability of cluster (i.e. probability of being an artefact caused by multipath or similar).
    0: invalid
    1: <25%
    2: 50%
    3: 75%
    4: 90%
    5: 99%
    6: 99.9%
    7: <=100%
    """
    #RADAR - Read pcd from nuscenes devkit code
    meta = []
    with open(radar_file, 'rb') as f:
        for line in f:
            line = line.strip().decode('utf-8')
            meta.append(line)
            if line.startswith('DATA'):
                break

        data_binary = f.read()

     # Get the header rows and check if they appear as expected.
    assert meta[0].startswith('#'), 'First line must be comment'
    assert meta[1].startswith('VERSION'), 'Second line must be VERSION'
    #Get the fields of the data
    fields = meta[2].split(' ')[1:]
    sizes = meta[3].split(' ')[1:]
    types = meta[4].split(' ')[1:]
    counts = meta[5].split(' ')[1:]
    width = int(meta[6].split(' ')[1])
    height = int(meta[7].split(' ')[1])
    data = meta[10].split(' ')[1]
    feature_count = len(types)
    assert width > 0
    assert len([c for c in counts if c != c]) == 0, 'Error: COUNT not supported!'
    assert height == 1, 'Error: height != 0 not supported!'
    assert data == 'binary'

    types# Lookup table for how to decode the binaries.
    unpacking_lut = {'F': {2: 'e', 4: 'f', 8: 'd'},
                        'I': {1: 'b', 2: 'h', 4: 'i', 8: 'q'},
                        'U': {1: 'B', 2: 'H', 4: 'I', 8: 'Q'}}
    types_str = ''.join([unpacking_lut[t][int(s)] for t, s in zip(types, sizes)])

    # Decode each point.
    offset = 0
    point_count = width
    points = []
    for i in range(point_count):
        point = []
        for p in range(feature_count):
            start_p = offset
            end_p = start_p + int(sizes[p])
            assert end_p < len(data_binary)
            point_p = struct.unpack(types_str[p], data_binary[start_p:end_p])[0]
            point.append(point_p)
            offset = end_p
        points.append(point)

    # A NaN in the first point indicates an empty pointcloud.
    point = np.array(points[0])
    if np.any(np.isnan(point)):
        return np.zeros((feature_count, 0))

    # Convert to numpy matrix.
    points = np.array(points).transpose()

    # Class-level settings for radar pointclouds, see from_file().
    invalid_states = [0]  # type: List[int]
    dynprop_states = range(7)  # type: List[int] # Use [0, 2, 6] for moving objects only.
    ambig_states = [3]  # type: List[int]

     # If no parameters are provided, use default settings.
    invalid_states = p_invalid_states if invalid_states is None else invalid_states
    dynprop_states = p_dynprop_states if dynprop_states is None else dynprop_states
    ambig_states = p_ambig_states if ambig_states is None else ambig_states

    # Filter points with an invalid state. - 0 is an invalid state
    valid = [p in invalid_states for p in points[-4, :]]
    points = points[:, valid]

    # Filter by dynProp. - 0 - 6 are moving, 7 is stopped
    valid = [p in dynprop_states for p in points[3, :]]
    points = points[:, valid]

    # Filter by ambig_state. - Make sure Doppler ambiguity is unambiguous
    valid = [p in ambig_states for p in points[11, :]]
    points = points[:, valid]



    return points, fields

In [ ]:
#Returns np array of all radar points in ego frame
def fuse_radars_in_ego(sample):
    channels = ['RADAR_FRONT', 'RADAR_FRONT_LEFT', 'RADAR_FRONT_RIGHT', 'RADAR_BACK_LEFT', 'RADAR_BACK_RIGHT']
    #Merged points in ego frame
    radar_data =np.empty((18, 0))

    for channel in channels:
        #Load Radar in its own frame
        radar = nusc.get('sample_data', sample['data'][channel])
        radar_file = nusc.get_sample_data_path(radar['token'])
        scan, fields = radar_from_file(radar_file) #scan 18 x N
        # Transform radar points into coordinate frame of ego
        #Get Lidar Pose
        radar_sensor = nusc.get('calibrated_sensor', radar['calibrated_sensor_token'])
        radar_t = np.asarray(radar_sensor['translation'])
        radar_r_quat = Quaternion(radar_sensor['rotation'])

        #Rotate and translate
        scan[:3, :] = np.dot(radar_r_quat.rotation_matrix, scan[:3, :])
        scan[:3, :] = (scan[:3, :].transpose() + radar_t).transpose()

        radar_data = np.append(radar_data, scan, axis=1)
    return radar_data, fields

In [ ]:
#Show video of selected camera with lidar rendered into the image
def visualize_radar_in_img(scene, cam_channel='CAM_FRONT'):
    first_sample = nusc.get('sample', scene['first_sample_token'])
    last_sample = nusc.get('sample', scene['last_sample_token'])

    winname = cam_channel
    window_width = nusc.get('sample_data', first_sample['data'][cam_channel])['width']
    window_height = nusc.get('sample_data', first_sample['data'][cam_channel])['height']
    cv2.namedWindow(winname, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(winname, window_width, window_height)

    #Load lidar and img data from scan
    sample = first_sample

    while True:
        if sample['token'] == last_sample['token']:
            sample = first_sample
        #Load camera image
        cam = nusc.get('sample_data', sample['data'][cam_channel])
        cam_file = os.path.join(nusc.dataroot, cam['filename'])
        cam_img = cv2.imread(cam_file)

        scan, fields = fuse_radars_in_ego(sample) #scan 18 x N
        points = scan.transpose()[:, :3] #Does not need to be converted to int - Needs to be Nx18 for o3d

        #Get Camera Pose and intrinsic parameters
        cam_sensor = nusc.get('calibrated_sensor', cam['calibrated_sensor_token'])
        cam_t = np.asarray(cam_sensor['translation'])
        cam_r_quat = Quaternion(cam_sensor['rotation'])
        cam_k = np.asarray(cam_sensor['camera_intrinsic'])

        #Transform points to image frame - Using Open3D transformations
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points)

        # #First transformation - Skip because Radar points are alread in ego frame
        # pcd.rotate(radar_r_quat.rotation_matrix, np.array([0, 0, 0]))
        # pcd.translate(radar_t)

        #Second transformation - From ego to camera frame
        pcd.translate(-cam_t)
        pcd.rotate(cam_r_quat.rotation_matrix.T, np.array([0, 0, 0]))

        #Project points into image plane
        view = np.copy(cam_k)
        points = np.asarray(pcd.points).T[:3, :]

        #Set color to depth value
        depth = points[2, :]

        #Prepare intrinsics matrix for homogenous transformation
        viewpad = np.eye(4)
        viewpad[:view.shape[0], :view.shape[1]] = view

        #Prepare points for homogenous transformation
        points = np.concatenate((points, np.ones((1, points.shape[1])))) #4 X N matrix - Homogenous

        #Project Points into image
        points = np.dot(viewpad, points)

        #Remove w
        points = points[:3, :]

        #Normalize along Z axis - Divide by depth
        points = points / points[2, :]

        # Remove points that are either outside or behind the camera. Leave a margin of 1 pixel for aesthetic reasons.
        # Also make sure points are at least 1m in front of the camera to avoid seeing the lidar points on the camera
        # casing for non-keyframes which are slightly out of sync.
        min_dist = 1.0
        mask = np.ones(depth.shape[0], dtype=bool)
        mask = np.logical_and(mask, depth > min_dist)
        mask = np.logical_and(mask, points[0, :] > 1)
        mask = np.logical_and(mask, points[0, :] < cam_img.shape[1] - 1)#cv image has x axis on dimension 1
        mask = np.logical_and(mask, points[1, :] > 1)
        mask = np.logical_and(mask, points[1, :] < cam_img.shape[0] - 1)
        points = points[:, mask]

        color_img = np.copy(depth)
        color_img = color_img[mask]

        color = np.zeros(points.shape).transpose()
        scan = scan[:, mask]
        for i in range(color.shape[0]):
            color[i] = [255, 0, 0] if scan[8, i] > 0 else [0, 0, 255]

        #Draw Radar points in image - Red for moving towards ego, blue for moving away from ego
        for i in range(points.shape[1]):
            p_loc = (int(np.rint(points[0, i])), int(np.rint(points[1, i])))
            #text = "vX: " + "{:.5f}".format(scan[8, i]) + ", vY: " + "{:.5f}".format(scan[9, i])
            cv2.circle(cam_img, p_loc, 4, color[i], -1)
            #cv2.putText(cam_img, text, p_loc, cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),thickness=1)
        # i = int(points.shape[1] / 4)
        # p_loc = (int(np.rint(points[0, i])), int(np.rint(points[1, i])))
        # text = "vX: " + "{:.5f}".format(scan[8, i]) + ", vY: " + "{:.5f}".format(scan[9, i])
        # cv2.circle(cam_img, p_loc, 4, color[i], -1)
        # cv2.putText(cam_img, text, p_loc, cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),thickness=1)

        cv2.imshow(winname, cam_img)
        key = cv2.waitKey(400)
        if key == 32:
            key = 0
            while (key != 32 and key != ord('q') and key != 27): #Space bar
                key = cv2.waitKey(0)

        if key == ord('q') or key == 27:
            break

        sample = nusc.get('sample', sample['next'])
    cv2.destroyAllWindows()

# End RADAR

In [ ]:
#visualize_lidar_in_img(scene, cam_channel='CAM_FRONT')
#colorise_pcd(scene)

In [ ]:
#Show the map of the scene
#view_scene_map(scene)
#visualize_lidar_in_img(scene, cam_channel='CAM_FRONT')
#colorise_pcd(scene)
#cam_vis(scene, "CAM_BACK")
#interactive_vis(scene, record=True)
#top_down(scene, record=True)

In [ ]:
#Show front cam view through scene
#cam_vis(scene, "CAM_BACK")

### Show Pointcloud LIDAR

In [ ]:
#Visualize with interactive view control
#interactive_vis(scene, record=True)

In [ ]:
#Visualize top down view only
#top_down(scene, record=True)

### Fuse colored pointcloud

In [ ]:
first_sample = nusc.get('sample', scene['first_sample_token'])
last_sample = nusc.get('sample', scene['last_sample_token'])
sample = first_sample

In [ ]:
first_sample = nusc.get('sample', scene['first_sample_token'])
last_sample = nusc.get('sample', scene['last_sample_token'])
sample = first_sample
channels = ['CAM_FRONT', 'CAM_FRONT_LEFT', 'CAM_FRONT_RIGHT', 'CAM_BACK', 'CAM_BACK_LEFT', 'CAM_BACK_RIGHT']

pcd = o3d.geometry.PointCloud()
#Visualize a video sequence of LIDAR data  

#Render callback for non blocking visualization which allows smoother control of view
#Define the callback here to simplify the transfer of variables samples, and pcd
def render_callback(vis):
    global sample, pcd, channels
    cam_channel = 'CAM_FRONT_LEFT'
    if sample['token'] == last_sample['token']:
        return False

    #Load next scan
    #cam = nusc.get('sample_data', sample['data'][cam_channel])
    #Load lidar scan
    lidar = nusc.get('sample_data', sample['data']['LIDAR_TOP'])

    #Merged points
    pcd_points =np.empty((0, 3))
    pcd_colors =np.empty((0, 3))
    for channel in channels:
        #Load next scan
        cam = nusc.get('sample_data', sample['data'][channel])
        points, color = project_points_in_img(cam, lidar)
        pcd_points = np.append(pcd_points, points, axis=0)
        pcd_colors = np.append(pcd_colors, color, axis=0)

    #pcd_points, pcd_color = project_points_in_img(cam, lidar)
    #####
    print("FINAL POINTS: ", pcd_points.shape)
    lidar_file = nusc.get_sample_data_path(lidar['token'])
    scan = np.fromfile(lidar_file, dtype=np.float32).reshape(-1, 5) #x, y, z, intensity, Ring Index
    points = scan[:, :3]
    #print("ORIG Points: ", points.shape)
    #print("Percentage: ", pcd_points.shape[0] / points.shape[0])
    #####
    #Update Geometry
    if sample['token'] == first_sample['token']:
        pcd.points = o3d.utility.Vector3dVector(pcd_points)
        pcd.colors = o3d.utility.Vector3dVector(pcd_colors)
        vis.add_geometry(pcd)
    pcd.points = o3d.utility.Vector3dVector(pcd_points) #Only need XYZ
    pcd.colors = o3d.utility.Vector3dVector(pcd_colors)
    #vis.clear_geometries()
    #vis.add_geometry(pcd)

    # Update the visualization window and process events to handle keyboard inputs.
    vis.update_geometry(pcd)
    vis.poll_events()
    vis.update_renderer()

    #Next Sample
    sample = nusc.get('sample', sample['next'])

    #Slow down animation
    time.sleep(0.25)

    return True


# Create an Open3D visualization window.
vis = o3d.visualization.VisualizerWithKeyCallback()

# Set the custom update function.
vis.register_animation_callback(render_callback)


# Start animation.
vis.create_window()

opt = vis.get_render_option()
opt.background_color = np.asarray([0, 0, 0])
opt.point_size = 2.5 #Original 5

vis.run()
vis.destroy_window()

In [ ]:
pcd_points =np.empty((3, 0))

In [ ]:
np.append(pcd_points, np.array([[1, 1], [2, 2], [3, 3] ]), axis=1)

### LIDAR 2 IMG Projection

In [ ]:
#Load lidar and img data from scan
sample = first_sample
cam_channel = 'CAM_FRONT_RIGHT'
#Load camera image
cam = nusc.get('sample_data', sample['data'][cam_channel])
cam_file = os.path.join(nusc.dataroot, cam['filename'])
cam_img = cv2.imread(cam_file)

#Load lidar scan
lidar = nusc.get('sample_data', sample['data']['LIDAR_TOP'])
lidar_file = nusc.get_sample_data_path(lidar['token'])
scan = np.fromfile(lidar_file, dtype=np.float32).reshape(-1, 5) #x, y, z, intensity, Ring Index
points = scan[:, :3] #Only x, y, z

#Get Lidar Pose
lidar_sensor = nusc.get('calibrated_sensor', lidar['calibrated_sensor_token'])
lidar_t = np.asarray(lidar_sensor['translation'])
lidar_r_quat = Quaternion(lidar_sensor['rotation'])

#Get Camera Pose and intrinsic parameters
cam_sensor = nusc.get('calibrated_sensor', cam['calibrated_sensor_token'])
cam_t = np.asarray(cam_sensor['translation'])
cam_r_quat = Quaternion(cam_sensor['rotation'])
cam_k = np.asarray(cam_sensor['camera_intrinsic'])

#Transform points to image frame - Using Open3D transformations
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

#First transformation
pcd.rotate(lidar_r_quat.rotation_matrix, np.array([0, 0, 0]))
pcd.translate(lidar_t)

#Second transformation
pcd.translate(-cam_t)
pcd.rotate(cam_r_quat.rotation_matrix.T, np.array([0, 0, 0]))

#Project points into image plane
view = np.copy(cam_k)
points = np.asarray(pcd.points).T[:3, :]

#Set color to depth value
depth = points[2, :]

#Prepare intrinsics matrix for homogenous transformation
viewpad = np.eye(4)
viewpad[:view.shape[0], :view.shape[1]] = view

#Prepare points for homogenous transformation
points = np.concatenate((points, np.ones((1, points.shape[1])))) #4 X N matrix - Homogenous

#Project Points into image
points = np.dot(viewpad, points)

#Remove w
points = points[:3, :]

#Normalize along Z axis - Divide by depth
points = points / points[2, :]

# Remove points that are either outside or behind the camera. Leave a margin of 1 pixel for aesthetic reasons.
# Also make sure points are at least 1m in front of the camera to avoid seeing the lidar points on the camera
# casing for non-keyframes which are slightly out of sync.
min_dist = 1.0
mask = np.ones(points.shape[1], dtype=bool)
mask = np.logical_and(mask, depth > min_dist)
mask = np.logical_and(mask, points[0, :] > 1)
mask = np.logical_and(mask, points[0, :] < cam_img.shape[1] - 1)#cv image has x axis on dimension 1
mask = np.logical_and(mask, points[1, :] > 1)
mask = np.logical_and(mask, points[1, :] < cam_img.shape[0] - 1)
points_img = np.uint8(np.rint(points))[:2, mask] #The x, y coordinates of the points that lie in the image
points = points[:, mask]

#Extract color values from image
pts_color = np.zeros((points.shape[1], 3))
for i in range(points.shape[1]):
    pts_color[i] = cam_img[int(np.rint(points[1, i])), int(np.rint(points[0, i]))]
#pts_color = (pts_color - pts_color.min()) / (pts_color.max() - pts_color.min()) #Normalize to range 0 to 1

#BGR to RGB
pts_color = pts_color[:, ::-1]
pts_color = pts_color.astype(np.float32) / 255.0

pcd_points = scan[mask, :3]
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pcd_points) #Nx3 #pcd_points
pcd.colors = o3d.utility.Vector3dVector(pts_color) #Nx3

#o3d.visualization.draw_geometries([pcd])
custom_draw_geometry(pcd)

In [ ]:
pcd_points = scan[mask, :3]
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pcd_points) #Nx3 #pcd_points
pcd.colors = o3d.utility.Vector3dVector(pts_color) #Nx3

#o3d.visualization.draw_geometries([pcd])
custom_draw_geometry(pcd)

In [ ]:

winname = cam['channel']
window_width, window_height = 800, 600
cv2.namedWindow(winname, cv2.WINDOW_NORMAL)
cv2.resizeWindow(winname, window_width, window_height)

cv2.imshow(winname, cam_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cv2.imwrite('output_image.jpg', cam_img)


In [ ]:
cam_img.shape